In [1]:

import pandas as pd
import re
import numpy as np
import time
import collections
from nltk.corpus import stopwords
import nltk.data
import networkx as nx
import proj_base
from gensim.models import word2vec
import matplotlib.pyplot as plt
#tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [21]:
import scipy.spatial.distance as scpd

In [2]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [3]:
data = proj_base.getBlankFrame()
#data = proj_base.addFileToData('./Review_Texts/hotel_100504.dat', data, ) 
data = proj_base.addFileToData('./Review_Texts/hotel_72572.dat', data) 


In [4]:
nodes = data.shape[0]
print(nodes)
data.head()
#len(set(data["Author"]))

75


,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Jan 6, 2009",5.0,5.0,5.0,everywhereman2,5.0,NaN,5.0,old seattle getaway this was old world excelle...,5.0,NaN,5.0
1,5.0,"Dec 14, 2008",5.0,NaN,4.0,KGBT,4.0,NaN,5.0,"wow, what charm! as a travel agent, i've staye...",4.0,NaN,5.0
2,4.0,"Dec 1, 2008",5.0,5.0,5.0,Marilyn1949,5.0,NaN,5.0,great location for sporting events we attend s...,4.0,NaN,4.0
3,5.0,"Nov 28, 2008",5.0,5.0,2.0,MissViolet105,5.0,NaN,5.0,pioneer square bw fine choice we chose bw pion...,5.0,NaN,4.0
4,5.0,"Nov 25, 2008",5.0,5.0,5.0,fallriverma,5.0,NaN,5.0,quality hotel at great price very clean. free ...,5.0,NaN,5.0


In [5]:
sentences = []

#word2vec takes a list of lists, where each internal list is a BOW


def review2sentences(review, tokenizer, remove_stopwords=True):
    #split each review into sentences
    raw_sent = tokenizer.tokenize(review.strip())
    
    sentences = []
    for s in raw_sent:
        if len(s) > 0 :
            sentences.append(review2wordlist(s, remove_stopwords))
            
    return sentences


def review2wordlist(review_text, remove_stopwords=True):
    #split the given text into BOW
    

    review_text = re.sub("[^a-zA-Z]"," ", review_text)

    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return(words)


for review in data["Content"]:
    sentences += review2sentences(review, tokenizer)
    
    

In [19]:
#BUILD WORD2Vec infrastructure

num_features = 100
min_word_count = 40
num_workers = 2
context = 2
downsampling = 1e-3

from gensim.models import word2vec
print("training")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features)

def makeFeatureVec(words, model, num_features):
    #given a list of words and the pre-trained word model return a feature vector
    featureVec = np.zeros((num_features,), dtype="float32")
    
    nwords = 0
    index2word_set = set(model.index2word)
    
    
    #get the mean vector for each review
    for w in words:
        #print(w)
        if w in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[w])
            
    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVec(reviews, model, num_features):
    #get average feature vec for list of bag of words
    count = 0
    reviewFeatureVec = np.zeros((len(reviews), num_features), dtype="float32")
    
    for r in reviews:
        if count % 1000 == 0:
            print("at review", count)
        #for each review add the feature vec
        reviewFeatureVec[count] = makeFeatureVec(r, model, num_features)
        count += 1
        
    return reviewFeatureVec




training


In [20]:
rev1 = data["Content"][0]
rev12words = review2wordlist(rev1, tokenizer)
print(rev12words)
rev1avg = makeFeatureVec(rev12words, model, num_features)
print(rev1avg)

['old', 'seattle', 'getaway', 'old', 'world', 'excellence', 'best', 'place', 'stay', 'visiting', 'historical', 'area', 'seattle', 'right', 'water', 'front', 'near', 'ferry', 'great', 'sea', 'food', 'restraunts', 'still', 'walking', 'distance', 'great', 'blues', 'jazz', 'music', 'staff', 'hotel', 'excellent', 'make', 'feel', 'right', 'home', 'breakfast', 'great', 'nt', 'travel', 'far', 'good', 'cup', 'joe', 'light', 'meal', 'start', 'adventurous', 'day', 'one', 'beautifull', 'city', 'america', 'hotel', 'area', 'makes', 'easy', 'get', 'place', 'want', 'go', 'still', 'find', 'way', 'back', 'highly', 'recomend', 'hotel', 'next', 'visit', 'seattle']
[-0.00182859  0.0010434  -0.00205694  0.00189076 -0.00419966 -0.00039751
  0.00052745  0.00035069 -0.00148416 -0.00143551 -0.00164039 -0.00111382
  0.00625432  0.00043604 -0.0015831  -0.00059523  0.00135195 -0.0031035
  0.00150067 -0.00045331  0.00095373 -0.0003382   0.00205024 -0.00042875
 -0.00104579 -0.00227782  0.00237869  0.0018574   0.0011

In [23]:
#GET HARD TRUTH RATINGS

aspect="Location"

def filterHardTruthForAspect(data, aspect):
    #give data points that have hard truth ratings
    return  data[pd.notnull(data[aspect])]

justLoc = filterHardTruthForAspect(data, aspect)
justLoc.shape

(46, 13)

In [25]:

def buildGraph(data, sim):
    
    G = nx.Graph()
    neighborsForNodes = 6
    nodes = data.shape[0]
    # build nodes
    for i in range(nodes):
        rev = data.loc[i]
        content = rev["Content"]
        #todo only add relevant sentences or take whole content?
        
        G.add_node(i, {'author' : rev["Author"], 'content' :content})
        
        #if i has a hard-truth rating add a node with that dongle
        #if rev[aspect] != np.nan:
        #    G.add_node(i+nodes, truth=True, rating=rev[aspect])
        #    G.add_edge(i, i+nodes, weight=1)
    
    # build edges
    sims = buildSimilarityMatrix(data)
    for i in range(nodes):
        for j in range(nodes):
            G.add_edge(i, j, weight=sims[i][j])
    
    

    # add separate learner scores
    
    return G

def howSimilar(r1, r2):
    #use word2vec cosine similarity
    
    rev2words = review2wordlist(r1, tokenizer)
    rev1vec = makeFeatureVec(rev2words, model, num_features)
    revj2words = review2wordlist(r2)
    rev2vec = makeFeatureVec(revj2words, model, num_features)
    return scpd.cosine(rev1vec, rev2vec)


def buildSimilarityMatrix(data):
    
    similar = [[0]*nodes for i in range(nodes)]
    
    for i in range(nodes):
        dati = data.loc[i]["Content"]
        
        for j in range(nodes):
            if i != j:
                datj = data.loc[j]["Content"]
               
                similar[i][j] = howSimilar(dati, data.loc[j]["Content"])
                
    return similar


def getRatings(graph):
    #minimize loss ratings over the graph
    
    
    #for edge in graph
    #   loss = 
    return



similarities = buildSimilarityMatrix(data)
g = buildGraph(data, similarities)

In [29]:
rev2words = review2wordlist(data["Content"][0], tokenizer)
rev1vec = makeFeatureVec(rev2words, model, num_features)
revj2words = review2wordlist(data["Content"][1])
rev2vec = makeFeatureVec(revj2words, model, num_features)
scpd.cosine(rev1vec, rev2vec)

0.045377495137383539

In [31]:
#nx.get_edge_attributes(g, 'weight')

In [ ]:
%matplotlib inline
nx.draw(g)
plt.show()

In [ ]:
data.shape

In [ ]:
sentences[0:3]

In [ ]:
#from sklearn.svm import SVC

#do svm classification
#need to split data into hard-truth train and predict
#not sure how were actually going to do this. Hard-truth ratings are needed to train the model right?
#but we'll also need to have some to test the results
#so probably going to need to have a lot of reviews

#train = sample of data with hard rating

trainDataVecs = getAvgFeatureVec(train, model, num_features)
